# <div style='color:white;background: #005792;text-align: center;padding: 15px 0'>Recommandations - Transformation des donnees Movielens</div>

## Participants
* Samantha
* Rachelle
* Andrew

## <div style='background: #005792;text-align: center;padding: 15px 0'> <a style= 'color:white;' >Configuration des variables globales</a></div>

### Importation des librairies

In [1]:
import pandas as pd

### Chargement des fichiers

In [ ]:
source_dir= '/home/dstrec/dstrec/010_data/000_source/movielens'

file_genome_tags = f"{source_dir}/genome-tags.csv"
file_genome_scores = f"{source_dir}/genome-scores.csv"
file_tags = f"{source_dir}/tags.csv"
file_ratings = f"{source_dir}/ratings.csv"
file_movies = f"{source_dir}/movies.csv"
file_links = f"{source_dir}/links.csv"

### Chargement des fichiers

In [2]:
genome_tags = pd.read_csv(file_genome_tags)
genome_scores = pd.read_csv(file_genome_scores)
tags = pd.read_csv(file_tags)
ratings = pd.read_csv(file_ratings)
movies = pd.read_csv(file_movies)
links = pd.read_csv(file_links)

## <div style='background: #005792;text-align: center;padding: 15px 0'> <a style= 'color:white;' >Préparation des données</a></div>

### Fichiers Genome-scores et Genome-tags

In [3]:
#Nous choississons de ne pas tenir compte des datasets genome-scores et genome-tags
#qui ne seront pas utiles dans notre projet et ne seront pas relié à nos données imdb


### Fichier Movies

In [4]:
#Nettoyage du dataset movies:

#Suppression des années dans les titres des films:
movies['title'] = movies['title'].str.replace(r' \(\d{4}\)', '', regex=True)

movies.head()

,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy


### Fichier Links

In [5]:
#Nettoyage du dataset links:

#Suppression dans le dataframe links de la colonne tmdbId qui comporte des 
#valeurs manquantes et qui ne nous servira pas pour la suite

links.drop('tmdbId', axis=1, inplace=True)

#Ajout de "tt0" devant les valeurs de la colonne imdbI:
links['imdbId'] = links['imdbId'].apply(lambda x: 'tt0' + str(x))

links.head()

,movieId,imdbId
0,1,tt0114709
1,2,tt0113497
2,3,tt0113228
3,4,tt0114885
4,5,tt0113041


### Fichier Ratings

In [6]:
#Nettoyage du dataset ratings:

#Suppression de la colonne timestamp qui ne nous servira pas pour la suite 
ratings.drop('timestamp', axis=1, inplace=True)

In [7]:
#Remplacer la colonne rating par la moyenne des notes obtenues pour chaque film
average_ratings = ratings.groupby('movieId')['rating'].mean().reset_index()
average_ratings.rename(columns={'rating': 'average_rating'}, inplace=True)

# Fusionner les moyennes de notes avec le DataFrame original ratings
ratings = pd.merge(ratings, average_ratings, on='movieId', how='left')

# Supprimer la colonne 'rating' originale
ratings.drop(columns=['rating'], inplace=True)

#Arrondir la note obtenu à une décimale
ratings['average_rating'] = ratings['average_rating'].round(1)

ratings.head()

,userId,movieId,average_rating
0,1,2,3.2
1,1,29,4.0
2,1,32,3.9
3,1,47,4.1
4,1,50,4.3


In [8]:
#Supprimer la colonne userId
ratings.drop('userId', axis=1, inplace=True)

#Suppression des doublons
ratings.drop_duplicates(inplace=True)

ratings.head()

,movieId,average_rating
0,2,3.2
1,29,4.0
2,32,3.9
3,47,4.1
4,50,4.3


### Fichier Tags

In [9]:
#Nettoyage du dataset tags:

#Suppression de la colonne timestamp qui ne nous servira pas 
tags.drop('timestamp', axis=1, inplace=True)

In [10]:
#Suppression des lignes contenant des valeurs manquantes dans la colonne tag
tags.dropna(subset=['tag'], inplace=True)

In [11]:
#Remplacement de la valeur de tag en fonction du tag qui ressort le plus pour chaque film
most_common_tags = tags.groupby('movieId')['tag'].apply(lambda x: x.value_counts().idxmax()).reset_index()
most_common_tags.rename(columns={'tag': 'most_common_tag'}, inplace=True)

# Fusionner les tags les plus courants avec le DataFrame original tags
tags = pd.merge(tags, most_common_tags, on='movieId', how='left')

# Supprimer la colonne 'tag' originale
tags.drop(columns=['tag'], inplace=True)

tags.head()

,userId,movieId,most_common_tag
0,18,4141,Mark Waters
1,65,208,post-apocalyptic
2,65,353,dark hero
3,65,521,noir thriller
4,65,592,dark hero


In [12]:
#Suppression de la colonne userId
tags.drop('userId', axis=1, inplace=True)

#Suppression des doublons
tags.drop_duplicates(inplace=True)

tags.head()

,movieId,most_common_tag
0,4141,Mark Waters
1,208,post-apocalyptic
2,353,dark hero
3,521,noir thriller
4,592,dark hero


### Fusion des fichiers

In [13]:
#Fusion des différents datasets nettoyés via movieId et détermination de l'index du dataframe sur movieId
movielens = pd.merge(movies, links, on='movieId')
movielens = pd.merge(movielens, ratings, on='movieId')
movielens = pd.merge(movielens, tags, on='movieId')

movielens.set_index('movieId', inplace=True)

movielens.head(15)

,title,genres,imdbId,average_rating,most_common_tag
movieId,,,,,
1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,tt0114709,3.9,Pixar
2,Jumanji,Adventure|Children|Fantasy,tt0113497,3.2,Robin Williams
3,Grumpier Old Men,Comedy|Romance,tt0113228,3.2,moldy
4,Waiting to Exhale,Comedy|Drama|Romance,tt0114885,2.9,chick flick
5,Father of the Bride Part II,Comedy,tt0113041,3.1,steve martin
6,Heat,Action|Crime|Thriller,tt0113277,3.8,Al Pacino
7,Sabrina,Comedy|Romance,tt0114319,3.4,remake
8,Tom and Huck,Adventure|Children,tt0112302,3.1,adapted from:book
9,Sudden Death,Action,tt0114576,3.0,Jean-Claude Van Damme


In [14]:
movielens.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19011 entries, 1 to 131258
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            19011 non-null  object 
 1   genres           19011 non-null  object 
 2   imdbId           19011 non-null  object 
 3   average_rating   19011 non-null  float64
 4   most_common_tag  19011 non-null  object 
dtypes: float64(1), object(4)
memory usage: 891.1+ KB


In [15]:
movielens.shape

(19011, 5)

### Vérification des doublons

In [16]:
doublons_title = movielens.duplicated(subset=['title'])
print("Nombre de doublons dans 'title':", doublons_title.sum())

print(movielens[doublons_title].sort_values(by='title'))

Nombre de doublons dans 'title': 659
                                title                      genres      imdbId  \
movieId                                                                         
77846                    12 Angry Men                 Crime|Drama   tt0118528   
114130   20,000 Leagues Under the Sea              Romance|Sci-Fi   tt0118247   
102190   20,000 Leagues Under the Sea    Adventure|Romance|Sci-Fi   tt0118248   
62383    20,000 Leagues Under the Sea     Action|Adventure|Sci-Fi     tt06333   
54997                    3:10 to Yuma  Action|Crime|Drama|Western   tt0381849   
...                               ...                         ...         ...   
116261              Wuthering Heights                       Drama  tt01238834   
91035               Wuthering Heights                       Drama  tt01181614   
7179                Wuthering Heights               Drama|Romance   tt0104181   
40962            Yours, Mine and Ours              Comedy|Romance   tt04

In [17]:
movielens_clean = movielens.drop_duplicates(subset=['title'], keep='first')
print("Taille du dataframe après suppression des doublons:", movielens_clean.shape)

print("Avant suppression:", movielens.shape)
print("Après suppression:", movielens_clean.shape)

Taille du dataframe après suppression des doublons: (18352, 5)
Avant suppression: (19011, 5)
Après suppression: (18352, 5)


### Affichage de movielens

In [18]:
movielens.head()

,title,genres,imdbId,average_rating,most_common_tag
movieId,,,,,
1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,tt0114709,3.9,Pixar
2,Jumanji,Adventure|Children|Fantasy,tt0113497,3.2,Robin Williams
3,Grumpier Old Men,Comedy|Romance,tt0113228,3.2,moldy
4,Waiting to Exhale,Comedy|Drama|Romance,tt0114885,2.9,chick flick
5,Father of the Bride Part II,Comedy,tt0113041,3.1,steve martin


### Création du fichier movielens.csv

In [19]:
dest_dir = '/home/dstrec/dstrec/010_data/001_transformed'

output_file_movielens_clean = f"{dest_dir}/movielens.csv"

movielens_clean.to_csv(output_file_movielens_clean, index=False)